In [136]:
import dgl
import torch
import torch.nn as nn
from dgl.nn import MetaPath2Vec
from openhgnn.dataset.NodeClassificationDataset import OHGB_NodeClassification
from torch.optim import SparseAdam
from torch.utils.data import DataLoader
from tqdm import tqdm


acm = OHGB_NodeClassification(
    dataset_name="ohgbn-acm", raw_dir="./dataset", logger=None
)

Extracting file to ./openhgnn/dataset\ohgbn-acm
Done saving data into cached files.


In [2]:
hg = acm.g
meta_paths_dict = acm.meta_paths_dict

In [3]:
g_pap = dgl.metapath_reachable_graph(hg, meta_paths_dict["PAP"])

## test Mp2Vec

In [145]:
m2v_negative_size = 5
m2v_emb_dim = 128
m2v_window_size = 3
m2v_lr = 0.001

m2v_batch_size: 256
m2v_epoch= 20

# m2v_rw_walk_length= 10
# m2v_rw_walks_per_node= 3


In [146]:
m2v = MetaPath2Vec(hg, meta_paths_dict["PAP"], window_size=3)

100%|████████████████████████████████████████████████████████████████████████████| 3025/3025 [00:02<00:00, 1496.39it/s]


In [147]:
optimizer = SparseAdam(m2v.parameters(), lr=m2v_lr)

In [148]:
dataloader = DataLoader(
    torch.arange(hg.num_nodes("paper")),
    batch_size=128,
    shuffle=True,
    collate_fn=m2v.sample,
)

In [149]:
for _ in tqdm(range(m2v_epoch)):
    for (pos_u, pos_v, neg_v) in dataloader:
        loss = m2v(pos_u, pos_v, neg_v)
        optimizer.zero_grad()
        loss.backward()
    optimizer.step()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.68it/s]


In [150]:
# Get the embeddings of all user nodes
user_nids = torch.LongTensor(m2v.local_to_global_nid["paper"])
user_emb = m2v.node_embed(user_nids)